In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

urls = [
    "https://flo.health/menstrual-cycle/health/period/diy-menstrual-pads",
    "https://flo.health/menstrual-cycle/health/period/stress-and-period",
    "https://flo.health/menstrual-cycle/lifestyle/diet-and-nutrition/foods-to-eat-on-period",
    "https://flo.health/menstrual-cycle/menopause/symptoms/menopause-burning-sensation",
    "https://flo.health/menstrual-cycle/menopause/symptoms/perimenopause-periods-closer-together"
]

data = []

for url in urls:
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    titleElement = soup.find("h1", class_="flo-article-banner__title")
    title = titleElement.get_text(strip=True) if titleElement else "Title not found"
    
    publishedDate = "Published date not found"
    updatedDate = "Updated date not found"
    dateElements = soup.find_all("div", class_="flo-article-banner-bottom__info-panel-date--item")

    for element in dateElements:
        elementText = element.get_text()

        if "Published" in elementText:
            dateSpan = element.find("span")
            if dateSpan:
                publishedDate = dateSpan.get_text(strip=True)

        if "Updated" in elementText:
            dateSpan = element.find("span")
            if dateSpan:
                updatedDate = dateSpan.get_text(strip=True)

    sections = soup.find_all("section", class_="flo-article-text")
    allText = []

    for section in sections:
        sectionText = section.get_text(separator="\n", strip=True)
        allText.append(sectionText)

    finalText = "\n\n".join(allText)
    
    data.append([url, title, publishedDate, updatedDate, finalText])

df = pd.DataFrame(data, columns=["URL", "Title", "Published Date", "Updated Date", "Content"])

df.to_csv("articleData.csv", index=False)

